In [ ]:
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense      #neural net layer
import pandas as pd                  
from sklearn.model_selection import train_test_split   #to spit dataset into train,validation,test 

import tensorflow as tf


import numpy as np
from sklearn import metrics
import scipy as sp

def mse(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.square(np.subtract(actual,pred)).mean()                                                                                  

In [ ]:
import os
os.chdir('C:/Users/USER/Desktop/TheGoldenTrio/DCMD_transport_model')
os.getcwd()

In [ ]:
#Store the dataset
df = pd.read_excel(r"C:\Users\USER\Desktop\TheGoldenTrio\new_topsis.xlsx")

dataset = df.values                          #Convert the data into an array
inputs  = dataset[2:279,0:4]                #end - 1  
targets = dataset[2:279,4:5]               #end

In [ ]:
#How to split this into train & test data?
X_train, x_test, Y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state = 4)   
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.fit_transform(x_test)
Y_train = sc.fit_transform(y_train)
Y_test = sc.fit_transform(y_test)
INPUTS = sc.fit_transform(inputs)
TARGETS = sc.fit_transform(targets)
X_val = sc.fit_transform(x_val)
Y_val = sc.fit_transform(y_val)

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [ ]:
HP_1 = hp.HParam('num_units1', hp.Discrete([8,16,24,32,36,48,54,60,72,76,80,84,88,94,96])) #
HP_2 = hp.HParam('num_units2', hp.Discrete([8,16,24,32,36,48,54,60])) #4,8,12,16,20,24,32,36,
HP_3 = hp.HParam('num_units3', hp.Discrete([4,8,16,20])) #2,4,6,8,
HP_ACTIVATION_1 = hp.HParam('act_1', hp.Discrete(['relu','sigmoid','tanh'])) #
HP_ACTIVATION_2 = hp.HParam('act_2', hp.Discrete(['relu','sigmoid','tanh'])) #
HP_ACTIVATION_3 = hp.HParam('act_3', hp.Discrete(['relu','tanh'])) #
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['Adam'])) #,'RMSprop','Adagrad','Adamax'

'''
MSE = 'mse'
R2_all = 'R2_all'
R2_train = 'R2_train'
R2_test = 'R2_test'
R2_val = 'R2_val'
#logdir = 'C:/Users/USER/Desktop/TheGoldenTrio/DCMD_transport_model/logs/hparam_tuning'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_1, HP_ACTIVATION_1, HP_2, HP_ACTIVATION_2, HP_3, HP_ACTIVATION_3, HP_OPTIMIZER],
    metrics=[hp.Metric(MSE, display_name='MSE'),hp.Metric(R2_all, display_name='R2_all'),hp.Metric(R2_train, display_name='R2_train') 
            ,hp.Metric(R2_test, display_name='R2_test'),hp.Metric(R2_val, display_name='R2_val')], 
  )
'''

In [ ]:
from sklearn.metrics import r2_score
#from tensorflow.keras.callbacks import TensorBoard
#log_folder = 'C:/Users/USER/Desktop/TheGoldenTrio/DCMD_transport_model/logs/'
#tb_callback = TensorBoard(log_dir=log_folder)

def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    Input(shape=(4, )), 
    Dense(hparams[HP_1], activation=hparams[HP_ACTIVATION_1]),
    Dense(hparams[HP_2], activation=hparams[HP_ACTIVATION_2]), 
    Dense(hparams[HP_3], activation=hparams[HP_ACTIVATION_3]),  
    Dense(1, )
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='mse',
      metrics=['mae'],
  )

  model.fit(X_train, Y_train, epochs=100, validation_data = (X_val,Y_val),batch_size=2) #, callbacks=[tb_callback]

  y_pred = model.predict(INPUTS)
  y_pred_inv = sc.inverse_transform(y_pred)

  y_predtrain = model.predict(X_train)
  y_predtrain_inv = sc.inverse_transform(y_predtrain)

  y_predval = model.predict(X_val)
  y_predval_inv = sc.inverse_transform(y_predval)

  y_predtest = model.predict(X_test)
  y_predtest_inv = sc.inverse_transform(y_predtest)
  print(model.summary())

  r2_all = r2_score(targets, y_pred_inv)
  r2_test = r2_score(y_test, y_predtest_inv)
  r2_train = r2_score(y_train, y_predtrain_inv)
  r2_val = r2_score(y_val, y_predval_inv)
  print('r2_all:', r2_all)
  print('r2_train:', r2_train)
  print('r2_test:', r2_test)
  print('r2_val:', r2_val)
  accuracy = [0,0,0,0,0]
  accuracy[0] = mse(targets, y_pred_inv)
  accuracy[1] = r2_all
  accuracy[2] = r2_train
  accuracy[3] = r2_test
  accuracy[4] = r2_val
  return accuracy

In [ ]:
def run(run_dir, hparams):
  acc = train_test_model(hparams)
  
  #keys_hp = hparams.keys()

  return acc
  
  '''with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = acc[0]
    R2_all = acc[1] 
    R2_train = acc[2]
    R2_test = acc[3]
    R2_val = acc[4]
    tf.summary.scalar('loss', accuracy, step=1)
    tf.summary.scalar('R2_all',R2_all, step=2)
    tf.summary.scalar('R2_train',R2_train, step=3)
    tf.summary.scalar('R2_test',R2_test, step=4)
    tf.summary.scalar('R2_val',R2_val, step=5)'''

In [ ]:
#import tensorboard

In [ ]:
session_num = 0
keys = ['loss', 'R2_all','R2_train','R2_test','R2_val']
 
data = []
for num_units1 in HP_1.domain.values:
    for act_1 in HP_ACTIVATION_1.domain.values:
        for num_units2 in HP_2.domain.values:
            for act_2 in HP_ACTIVATION_2.domain.values:
                for num_units3 in HP_3.domain.values:
                    for act_3 in HP_ACTIVATION_3.domain.values:
                        for optimizer in HP_OPTIMIZER.domain.values:
                            hparams = {
                              HP_1: num_units1,
                              HP_ACTIVATION_1: act_1,
                              HP_2: num_units2,
                              HP_ACTIVATION_2: act_2,
                              HP_3: num_units3,
                              HP_ACTIVATION_3: act_3, 
                              HP_OPTIMIZER: optimizer  
                            }
                            run_name = "run-%d" % session_num
                            print('--- Starting trial: %s' % run_name)
                            print({h.name: hparams[h] for h in hparams})
                            acc = run('C:/Users/USER/Desktop/TheGoldenTrio/DCMD_transport_model/logs/hparam_tuning/' + run_name, hparams)
                            tf.keras.backend.clear_session()
                            dict = {}
                            for (i,j) in zip(keys, acc):
                                dict[i] = j
                            for i in hparams.keys():
                                dict[i] = hparams[i]
                            data.append(dict)
                            session_num += 1
data = pd.DataFrame.from_dict(data)

In [ ]:
data.to_excel("Adam.xlsx")

In [ ]:
'''
%load_ext tensorboard

%tensorboard --logdir C:/Users/USER/Desktop/TheGoldenTrio/DCMD_transport_model/logs/hparam_tuning
#%tensorboard --logdir logs/hparam_tuning '''